In [ ]:
# Copyright (c) 2023 Sophie Katz
#
# This file is part of Sophie's ML Monorepo.
#
# Sophie's ML Monorepo is free software: you can redistribute it and/or modify it under
# the terms of the GNU General Public License as published by the Free Software
# Foundation, either version 3 of the License, or (at your option) any later version.
#
# Sophie's ML Monorepo is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A
# PARTICULAR PURPOSE. See the GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License along with Sophie's
# ML Monorepo. If not, see <https://www.gnu.org/licenses/>.

# Introduction to the Keras Tuner

Tutorial URL: https://www.tensorflow.org/tutorials/keras/keras_tuner

Valid as of: 2023.05.02

## Imports

In [1]:
import tensorflow as tf
import keras_tuner as kt

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.10.1


# Loading the dataset

In [2]:
# Use prebuilt MNIST dataset
dataset_mnist = tf.keras.datasets.mnist

# Load data from dataset
(data_train, label_train), (data_test, label_test) = dataset_mnist.load_data()

# Normalize data
data_train = data_train / 255.0
data_test = data_test / 255.0

# Declare model builder

In [3]:
def build_model(hp):
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(
                hp.Int("units", min_value=32, max_value=512, step=32), activation="relu"
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10),
        ]
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

# Declare tuner

In [4]:
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="training",
    project_name="hyperparameter_tuning",
)

# Tune model

In [5]:
tuner.search(
    data_train,
    label_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)],
)

hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Hyperparameters: {hyperparameters}")

Trial 30 Complete [00h 01m 27s]
val_accuracy: 0.9632499814033508

Best val_accuracy So Far: 0.9796666502952576
Total elapsed time: 00h 15m 31s
INFO:tensorflow:Oracle triggered exit
Hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x0000029C95D2A040>
